This code converts the spreadsheets into a single dataframe and creates a dictionary that tags each column with keys that identify string and/or number quality. These tags include capitalization errors, consistency in nonalphanumeric notation, misspellings, variance in numerical digits, etc.

In [ ]:
#Import libraries
import pandas as pd
from enchant.checker import SpellChecker ## package 'pyenchant'
import re

In [ ]:
#Import Data
Data = pd.ExcelFile('inputData/EAR_Raw_2016.xlsx')

In [ ]:
#New variable for each spreadsheet
table1 = pd.read_excel(Data, "Table1_Final")
table2 = pd.read_excel(Data, "Table2_Final")
table3 = pd.read_excel(Data, "Table3_Final")
table4 = pd.read_excel(Data, "Table4_LWS")
table5 = pd.read_excel(Data, "Table5_SWS")

#Organize into list, specify key for joining
tables = [table1, table2, table3, table4, table5]
key = "PWSID"

In [ ]:
#This funtion merges each spreadsheet to the first sequential based on a given key.
#In .merge(), kwarg 'how = outer' indicates all rows are kept regardless of matching PWSID.
def outer_join(key, tables):
    first_table = tables[0]
    for i in range(len(tables)):
        if i > 0:
            first_table = first_table.merge(tables[i], on=key, how="outer")
    return first_table

In [ ]:
#Print the shapes of the spreadsheets for comparison
for i in range(len(tables)):
    print("Spreadsheet %s shape =" %i, tables[i].shape,'\n')

In [ ]:
#Print the shape of the newly merged dataframe.
j_tables = outer_join(key, tables)
print("Joined Spreadsheet shape =", j_tables.shape)

In [ ]:
# NOTES: Function works as expected. The number 6838 is concerning. There are 21 PWSIDs that
# do not appear in at least one spreadsheet.

In [ ]:
tables_df = pd.DataFrame(j_tables)

In [ ]:
def nonalpha(col):
    nondict = ["@","!","\$","\(|\)|-","_","\"","\'","\*","&","~+"]
    percarr = []
    for n in nondict:
        reg = re.compile(n)
        nonalphaList = list(filter(reg.search,col))
        percarr.append((1.0*len(nonalphaList))/len(col))
    if np.max(percarr) > 0.01:
        return "{0:.2f}".format(np.max(percarr))+" "+nondict[percarr.index(np.max(percarr))]

def spellcheck(col):
    chkr = SpellChecker("en_US")
    mispelledrows = []
    for index, row in enumerate(col):
        chkr.set_text(row)
        errarr = []
        for error in chkr:
            errarr.append(error.word)
        if len(errarr) != 0:
            mispelledrows.append(index)
    return mispelledrows

def capital(col):
    words = np.asarray(col)
    nocaps = [word for word in words if word.islower()]
    allcaps = [word for word in words if word.isupper()]
    mixcaps = [word for word in words if not word.islower() and not word.isupper()]
    caparr = [len(nocaps),len(allcaps),len(mixcaps)]
    max_index = caparr.index(max(caparr))
    caparr.remove(max(caparr))
    
    return [["No Caps","Allcaps","Mixed Caps"][max_index],sum(caparr)]
    

In [ ]:
import numpy as np

def is_num(a):
    if type(a) is int or type(a) is float:
        return True
    else:
       return False

def tagger(col):
    tagdict = {}
    cleancol = list(filter(None,col))
    if any(isinstance(elem, str) for elem in cleancol):
        strcol = list(map(str,cleancol))
        tagdict["Capitalization, #OtherCaps"] = capital(strcol)
        tagdict["Nonalpha"] = nonalpha(strcol)
        tagdict["Misspelled Rows"] = spellcheck(strcol)
        if any(elem.startswith(" ") for elem in strcol):
            tagdict["Leading Space Rows"] = np.where(list(map(lambda x : x.startswith(" "),strcol)))
        if any(is_num(elem) for elem in cleancol):
            tagdict["Num Rows among Strings"] = (1.0*np.sum(list(map(lambda x : is_num(x),cleancol))))/len(cleancol)
    else:
        tagdict["Number Variance"] = np.var(list(map(lambda x : len(str(x)),col)))
        tagdict["Number Mean"] = np.mean(list(map(lambda x : len(str(x)),col)))
    return tagdict

#Capitalization tag: What is the most common capitalization and how many rows have different caps?
#Nonalpha tag: What is the most common nonalphanumeric character (@,parentheses,dashes,etc.) and what fraction of rows possess it?
#Misspelled Rows tag: Which rows have misspelled words?
#Lead Space Rows tag: Which rows have leading spaces?
#Num Rows among Strings tag: Are there columns with cells that contain either numbers of strings? What is the ratio of number rows to string rows?
#Number Variance: What is the digit variance of number columns?
#Number mean: What is the digit mean of number columns?

In [ ]:
tables_df.apply(lambda x: tagger(x), axis = 0)